In [1]:
import numpy
import cv2
import random
import pickle
import sklearn
import tensorflow as tf

In [2]:
from image_preprocessing.scan import *
from image_preprocessing.helper import resize,get_sorted_contours_bounding_box

In [74]:
img = cv2.imread(r'C:\Users\Avinash\Desktop\New folder\OCR\test ocr\umbc_address.png')
try:
    img = document_warper(img)
except FourPointException:
    print("ntg")

ntg


In [66]:
print(img.shape)
gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

(764, 700, 3)


In [67]:
edged = edge_detection(img)
cnts,heirarchy = cv2.findContours(edged.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)


In [68]:
cnts,bounding_box = get_sorted_contours_bounding_box(cnts)
cv2.imshow('w',gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [69]:
chars=[]
boxs=[]
for i,c in enumerate(cnts):
    x,y,w,h = bounding_box[i]
    if (w>=10 and w<=150) and (h>=15 and h<=150):
        roi = gray[y:y+h,x:x+w]
        thresh = cv2.threshold(roi,0,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
        dims = random.randint(45,55)
        thresh = cv2.resize(thresh,(dims,dims),cv2.INTER_CUBIC)
        tH,tW = thresh.shape
        dX = int(max(0,128-tW)/2.0)
        dY = int(max(0,128-tH)/2.0)
        padded = cv2.copyMakeBorder(thresh,top=dY,bottom=dY,right=dX,left=dX,borderType=cv2.BORDER_CONSTANT,value=(255,255,255))
        padded = cv2.resize(padded,(128,128),cv2.INTER_CUBIC)
        padded = padded.astype('float32')/255.
        padded = np.expand_dims(padded,axis=-1)
        boxs.append((x,y,w,h))
        chars.append(padded)

In [8]:
from tensorflow.keras.models import load_model
import json

In [9]:
# thresh=cv2.resize(thresh,(32,32),cv2.INTER_CUBIC)
# cv2.imshow("s",thresh)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
with tf.device('/cpu:0'):
    nums = load_model('models/alex_net nums.h5')
    lowercase = load_model('models/incep lower.h5')

In [10]:
with tf.device('/gpu:1'):
    uppercase = load_model('models/incep upper.h5')

In [11]:
with open('final rf 0.89.sav','rb') as f:
    random_forest = pickle.load(f)

In [70]:
chars = np.array(chars,dtype='float32')

In [20]:
np.reshape(chars[0],).shape

(1, 128, 128, 1)

In [22]:
with open('classes/lower_classes.json','r') as f:
    lower_classes = json.load(f)
f.close()
with open('classes/upper_classes.json','r') as f:
    upper_classes = json.load(f)
f.close()

In [31]:
cint('30',16)

'0'

In [73]:
def rf_classify(rf_preds,uc,lc,nums,chars):
    letter_class = []
    probs = []
    for i,c in enumerate(chars):
        tmp = np.reshape(c,((1,)+c.shape))

        if(rf_preds[i]==0):
            
            upper_pred = uppercase.predict(tmp)
            lower_pred = lowercaase.predict(tmp)
            idx1,idx2 = np.argmax(upper_pred),np.argmax(lower_pred)
            final_class = chr(int(upper_classes[str(idx1)],16))
            prob = upper_pred[0][idx1]
            if(upper_pred[0][idx1]<lower_pred[0][idx2]):
                final_class = chr(int(lower_classes[str(idx2)],16))
                prob = lower_pred[0][idx2]
            print("class:{} uppProb:{:.2f} lowerProb:{:.2f}".format(final_class,upper_pred[0][idx1],lower_pred[0][idx2]))
        if(rf_preds[i]==1 or prob<0.5):

            preds = nums.predict(tmp)
            idx1 = np.argmax(preds)
            final_class = chr(48+idx1)
            prob = preds[0][idx1]
        letter_class.append(final_class)
        probs.append(prob)
    return letter_class,probs


In [71]:


rf_preds=random_forest.predict(chars.reshape((chars.shape[0],chars.shape[1]*chars.shape[2])))
# for c in chars:
#     cv2.imshow('s',c)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

# preds = model.predict(chars)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 170 out of 170 | elapsed:    0.0s finished


In [59]:
for i,c in enumerate(chars):
    print(rf_preds[i])
    cv2.imshow(str(rf_preds[i]),c.reshape(128,128))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0


In [75]:
labels,probs = rf_classify(rf_preds,uppercase,lowercaase,nums,chars)
for (label,(x,y,w,h),prob) in zip(labels,boxs,probs):
    # idx = int(np.argmax(pred))
    # prob = pred[idx]
    # idx = str(idx)
    # label = chr(int(classes[idx],16))

    print("[INFO] {} -- {:.2f}%".format(label,prob*100))
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.putText(img,label,(x-10,y-10),cv2.FONT_HERSHEY_SIMPLEX,1.2,(0,255,0),2)
cv2.imshow('final',img)
cv2.waitKey(0)
cv2.destroyAllWindows()


class:B uppProb:0.96 lowerProb:0.52
class:o uppProb:1.00 lowerProb:1.00
class:A uppProb:0.99 lowerProb:0.71
class:L uppProb:1.00 lowerProb:0.62
class:o uppProb:0.89 lowerProb:0.99
class:T uppProb:1.00 lowerProb:0.87
class:i uppProb:0.97 lowerProb:1.00
class:o uppProb:0.73 lowerProb:0.99
class:m uppProb:0.65 lowerProb:0.95
class:U uppProb:0.71 lowerProb:0.23
class:H uppProb:1.00 lowerProb:0.88
class:I uppProb:1.00 lowerProb:0.96
class:R uppProb:1.00 lowerProb:0.77
class:L uppProb:1.00 lowerProb:0.66
class:E uppProb:1.00 lowerProb:0.69
class:L uppProb:1.00 lowerProb:0.59
class:t uppProb:0.15 lowerProb:0.79
class:m uppProb:0.56 lowerProb:1.00
class:U uppProb:0.92 lowerProb:0.69
class:p uppProb:1.00 lowerProb:1.00
class:p uppProb:0.90 lowerProb:1.00
class:z uppProb:1.00 lowerProb:1.00
class:c uppProb:0.97 lowerProb:1.00
class:I uppProb:0.97 lowerProb:0.85
class:R uppProb:0.99 lowerProb:0.99
class:z uppProb:0.95 lowerProb:0.98
class:o uppProb:0.48 lowerProb:0.79
[INFO] B -- 96.37%
[INFO] o 

In [14]:
cv2.imwrite(r"C:\Users\Avinash\Desktop\New folder\OCR\test result\hello world.jpg",img)

True

In [4]:
lowercase = load_model('models/incep lower.h5')

In [5]:
path = r'C:\Users\Avinash\Tensor serving models'
path = os.path.join(path,'ocr models')
# os.makedirs(path)
path

'C:\\Users\\Avinash\\Tensor serving models\\ocr models'

In [6]:
lowercase.save(filepath = path+'/lc',save_format='tf')
nums.save(filepath=path+'/nums',save_format='tf')

INFO:tensorflow:Assets written to: C:\Users\Avinash\Tensor serving models\ocr models/lc\assets
INFO:tensorflow:Assets written to: C:\Users\Avinash\Tensor serving models\ocr models/nums\assets
